In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
from drivers.Agilent_PNA_L import *
from drivers.Yokogawa_GS200 import *
from drivers.KeysightAWG import *
from drivers.E8257D import MXG

In [3]:
from lib2.MeasurementResult import MeasurementResult

In [79]:
vna = Agilent_PNA_L("PNA-L2")
current_src = Yokogawa_GS210("GS210_3")
current_src.set_current_limits(-12e-3, 12e-3)
ro_awg = KeysightAWG("AWG3")
q_awg = KeysightAWG("AWG2")
# mxg = MXG("MXG")

In [99]:
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [6]:
from importlib import reload
import lib2

In [82]:
from lib import data_management as dm

In [83]:
if_freq = 50e6

calibrations = (dm.load_IQMX_calibration_database("TEST", -20))
ro_cal = calibrations.get(frozenset(dict(lo_power=0, ssb_power=-50, lo_frequency=7.047e9, 
                                      if_frequency=0, waveform_resolution=0.1).items()))
ro_pb = PulseBuilder(ro_cal)

calibrations = (dm.load_IQMX_calibration_database("TESTQ", -6))
q_cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-5, lo_frequency=7.9e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=0.1).items()))
q_pb = PulseBuilder(q_cal)

## Single-tone spectroscopy

In [9]:
# reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

#### No mixer 

In [ ]:
STS = SingleToneSpectroscopy("IV-anticrossing", "Xmon Al BMSTU S444 2", vna, "current", current_src.set_current, 0)
vna_parameters = {"bandwidth":250, "freq_limits":(7030e6, 7070e6), "nop":101, "power":-40, "averages":1}
STS.setup_control_parameters(vna_parameters, linspace(-1e-3, 1e-3, 100))
result = STS.launch()
result.visualize()

In [ ]:
result.visualize()

In [ ]:
result.save()

#### Through mixer

In [85]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100, 5).build())

In [111]:
STS = SingleToneSpectroscopy("IV-anticrossing-through-mixer", "Xmon Al BMSTU S444 2", vna, "current", current_src.set_current, 0)
vna_parameters = {"bandwidth":500, "freq_limits":(7030e6, 7070e6), "nop":101, "power":-20, "averages":1}
currents = linspace(0.1e-3, 2e-3, 100)
STS.setup_control_parameters(vna_parameters, currents)
result = STS.launch()
current_src.set_current(0)
result.visualize()

Started at:  Mon Apr 24 16:11:55 2017
Time left: 0 h 0 m 32.87 s, parameter value: 1.000e-04, average cycle time: 0.33 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 2.000e-03, average cycle time: 0.28 s            

In [31]:
result.visualize()

In [33]:
result.save()

## Two-tone spectroscopy

In [128]:
# reload(lib2.TwoToneSpectroscopyBase)
reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [87]:
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100,5).build())

##### Qubit spectrum

In [88]:
mxg.set_output_state("OFF")
TTS = FluxTwoToneSpectroscopy("VI-2tone", "Xmon Al BMSTU S444 2", vna, mxg, current_src.set_current)
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-20, "averages":1}
mw_src_parameters = {"power":-5, "frequency":1j}
mw_src_frequencies = linspace(7e9, 8.5e9, 100)
currents = linspace(0.507e-3, 1.57e-3, 100)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at qubit's sweet spot (1.04 mA)
Detected frequency is 7.04720 GHz, at 0.75 mU and -20.51 degrees


In [89]:
mxg.set_output_state("ON")
result = TTS.launch()

Started at:  Mon Apr 24 14:03:24 2017
Time left: 8 h 1 m 52.98 s, parameter value: 5.070e-04, average cycle time: 2.89 s            

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 1.570e-03, average cycle time: 0.17 s             

In [92]:
result.visualize()

In [49]:
result._name = 'VI-2tone_wut'

In [91]:
result.save()

##### Linewidth from excitation power

In [100]:
mxg.set_output_state("OFF")
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmon Al BMSTU S444 2", vna, mxg, current_src.set_current, mxg.set_power)
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-20, "averages":1}
mw_src_parameters = {"power":-1, "frequency":-1}
mw_src_frequencies = linspace(7.88e9, 7.92e9,200)
mw_src_powers = linspace(-20, 15, 41)
current = 1.04e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of 1.04 mA
Detected frequency is 7.04720 GHz, at 0.80 mU and -21.49 degrees


In [101]:
mxg.set_output_state("ON")
result = PTS.launch()

Started at:  Mon Apr 24 14:45:49 2017
Time left: 6 h 43 m 54.45 s, parameter value: -2.000e+01, average cycle time: 2.96 s           

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 1.500e+01, average cycle time: 0.15 s              

In [ ]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [106]:
result.visualize()

In [103]:
result.save()

In [ ]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [ ]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

##### AC Stark

In [129]:
mxg.set_output_state("OFF")
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2", vna, mxg, current_src.set_current, mxg.set_power)
vna_parameters = {"bandwidth":1000, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-20, "averages":100}
mw_src_parameters = {"power":-15, "frequency":-1}
mw_src_frequencies = linspace(7.8e9, 7.9e9, 100)
vna_powers = linspace(-40, 0, 31)
current = 1.04e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of 1.04 mA
Detected frequency is 7.04707 GHz, at 0.84 mU and -26.23 degrees


In [135]:
mxg.set_output_state("ON")
result = ASTS.launch()

Started at:  Mon Apr 24 16:43:50 2017


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 17.93 s, parameter value: -1.333e+00, average cycle time: 0.18 s             Trying to set too small value: 0
Time left: 0 h 0 m 0.0 s, parameter value: 0.000e+00, average cycle time: 0.18 s            

In [137]:
result.visualize()

In [136]:
result.save()

## Loading

In [ ]:
result = MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-powerscan")

In [ ]:
result.set_phase_units("deg")

In [ ]:
result.visualize()

## Measurement class testing

In [1]:
from importlib import reload

In [64]:
from Measurement_AD_1 import *

In [73]:
meas = Measurement(['mxg'])

mxg MXG


VisaIOError: VI_ERROR_ALLOC (-1073807300): Insufficient system resources to perform necessary memory allocation.

In [75]:
meas._mxg.visa_instr.close()

In [12]:
meas._vna2.get_power()

0.0

In [62]:
mxg.visa_instr.close()

In [59]:
Agilent_EXA

NameError: name 'Agilent_EXA' is not defined

In [ ]:
from drivers import Agilent_PNA_L

In [57]:
mxg

TypeError: __repr__ returned non-string (type TCPIPInstrument)

In [ ]:
import drivers

In [ ]:
drivers.

In [ ]:
from drivers import *

In [ ]:
ю

In [ ]:
./

In [ ]:
from drivers import *

In [ ]:
Agilent_EXA